# Economic Data Analysis with FRED API

# 1. Install  FRED API

In [1]:
!pip install fredapi

  Obtaining dependency information for fredapi from https://files.pythonhosted.org/packages/96/d4/f81fa9f67775a6a4b9e2cd8487239d61a9698cb2b9c02a5a2897d310f7a4/fredapi-0.5.1-py3-none-any.whl.metadata


In [2]:
!pip show fredapi

Name: fredapi
Version: 0.5.1
Summary: Python API for Federal Reserve Economic Data (FRED) from St. Louis Fed
Home-page: https://github.com/mortada/fredapi
Author: Mortada Mehyar
Author-email: 
License: 
Location: /opt/conda/lib/python3.10/site-packages
Requires: pandas
Required-by: 


## 2. Import libraries

In [3]:
# Data manipulation
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px


# Plot aesthetics
plt.style.use('fivethirtyeight')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']

# FRED API from kaggle secrets
from fredapi import Fred

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
fred_key = user_secrets.get_secret('fred-api')


## 3. Create FRED object